In [38]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import Error
pd.options.display.float_format = '{:,}'.format

In [0]:
DIR = '/content/gdrive/My Drive/Goodreads/data'
DIR_GENRE = '/content/gdrive/My Drive/Goodreads/data/genre/'

In [0]:
def load_data(file_name):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            if (count>12000000):
              data.append(d)
            
            if count % 200000 == 0:
                print(count, end=',\n')
            
            #if count == 12000000:
                #break # just for test
            
            # break if reaches the 100th line
            #if (head is not None) and (count > head):
                #break
    return data

In [0]:
reviews = load_data(os.path.join(DIR, 'goodreads_reviews_dedup.json.gz'))

200000,
400000,
600000,
800000,
1000000,
1200000,
1400000,
1600000,
1800000,
2000000,
2200000,
2400000,
2600000,
2800000,
3000000,
3200000,
3400000,
3600000,
3800000,
4000000,
4200000,
4400000,
4600000,
4800000,
5000000,
5200000,
5400000,
5600000,
5800000,
6000000,
6200000,
6400000,
6600000,
6800000,
7000000,
7200000,
7400000,
7600000,
7800000,
8000000,
8200000,
8400000,
8600000,
8800000,
9000000,
9200000,
9400000,
9600000,
9800000,
10000000,
10200000,
10400000,
10600000,
10800000,
11000000,
11200000,
11400000,
11600000,
11800000,
12000000,
12200000,
12400000,
12600000,
12800000,
13000000,
13200000,
13400000,
13600000,
13800000,
14000000,
14200000,
14400000,
14600000,
14800000,
15000000,
15200000,
15400000,
15600000,


In [0]:
df_reviews0 = pd.DataFrame(reviews)

In [0]:
df_reviews0.head()

,book_id,date_added,date_updated,n_comments,n_votes,rating,read_at,review_id,review_text,started_at,user_id
0,25262904,Sun Apr 19 11:03:40 -0700 2015,Sun Apr 19 11:04:45 -0700 2015,0,1,5,Sun Apr 19 11:04:45 -0700 2015,edca2b61b538d3af76bcea8b4cb6316e,"""The Para-Portage of Emily"" is an eerie, other...",,93f2b067a8cf269907e7b66b95b1d192
1,18132276,Sun Apr 19 11:02:04 -0700 2015,Sun Apr 19 11:02:56 -0700 2015,0,0,5,Sun Apr 19 11:02:56 -0700 2015,bb75143e243f85d70bc17b8e3b15fcf4,"""Spellbound in His Arms"" is the story of Jacki...",,93f2b067a8cf269907e7b66b95b1d192
2,25357784,Thu Apr 16 18:36:45 -0700 2015,Tue Mar 08 08:01:02 -0800 2016,0,1,5,Thu Apr 16 00:00:00 -0700 2015,ad326c1e1469a50e76142e7c5d1b61b1,I did not know what to expect in purchasing th...,,93f2b067a8cf269907e7b66b95b1d192
3,22514483,Thu Apr 16 16:46:30 -0700 2015,Thu Apr 16 17:15:22 -0700 2015,0,0,5,Thu Apr 16 00:00:00 -0700 2015,9e7d9499ac2218156363662e1ec68695,"This is the first book I read in Roberts' ""Imm...",,93f2b067a8cf269907e7b66b95b1d192
4,25154803,Thu Apr 16 16:08:44 -0700 2015,Thu Apr 16 16:14:06 -0700 2015,0,1,5,Thu Apr 16 16:14:06 -0700 2015,b3cda0b5e628b91e243c4900505fe992,After reading Sabrina Devonshire's 'French Kis...,,93f2b067a8cf269907e7b66b95b1d192


In [0]:
df_reviews = df_reviews0[['book_id','user_id','review_text']].copy()

In [0]:
df_reviews.shape

(3739967, 3)

In [0]:
df_reviews.head()

,book_id,user_id,review_text
0,25262904,93f2b067a8cf269907e7b66b95b1d192,"""The Para-Portage of Emily"" is an eerie, other..."
1,18132276,93f2b067a8cf269907e7b66b95b1d192,"""Spellbound in His Arms"" is the story of Jacki..."
2,25357784,93f2b067a8cf269907e7b66b95b1d192,I did not know what to expect in purchasing th...
3,22514483,93f2b067a8cf269907e7b66b95b1d192,"This is the first book I read in Roberts' ""Imm..."
4,25154803,93f2b067a8cf269907e7b66b95b1d192,After reading Sabrina Devonshire's 'French Kis...


In [0]:
testform = df_reviews['book_id']
testform = testform.drop_duplicates(keep='first')
print(testform.shape)

(925729,)


In [0]:
df_reviews = df_reviews.groupby('book_id').agg({'review_text': ', '.join}).reset_index()

In [0]:
df_reviews.shape

(925729, 2)

In [0]:
export_csv = df_reviews.to_csv (r'/content/gdrive/My Drive/Goodreads/data/df_reviews3.csv', index = None, header=True)

In [0]:
df_reviews1 = pd.read_csv(os.path.join(DIR, 'df_reviews1.csv'))
df_reviews2 = pd.read_csv(os.path.join(DIR, 'df_reviews2.csv'))
df_reviews3 = pd.read_csv(os.path.join(DIR, 'df_reviews3.csv'))

In [0]:
df_reviews = pd.concat([df_reviews1,df_reviews2,df_reviews3])

In [0]:
df_reviews.shape

(3400167, 2)

In [0]:
testform = df_reviews['book_id']
testform = testform.drop_duplicates(keep='first')
print(testform.shape)

(2080190,)


In [0]:
df_reviews.dtypes

book_id         int64
review_text    object
dtype: object

In [0]:
df_reviews['book_id'] = df_reviews['book_id']
df_reviews['review_text'] = df_reviews['review_text'].astype(str)

In [0]:
df_reviews = df_reviews.groupby('book_id').agg({'review_text': ' '.join}).reset_index()

In [0]:
df_reviews.review_text[0]

'One of the best books in the series I think., I absolutely loved the entire HP series., Almost over. I don\'t think i can handle the suspense of wondering what is going to happen., dY~dY~dY~dY~dY~ \n The best of all the seven books, It only gets better as the suspense builds in this terrific, mysterious tale of possible double-crossing and virtuousness that delivers the best emotional mix of good versus evil on both a meta- and micro- scale since the Allies versus the Axis in the Second World War., ok but i miss dumbledore!!!! :(, Definitely the least bad of all the books. Two reasons: the youtube video, and the insane new adverbs she managed to come up with. Consolingly., series of my childhood!, First of all, Im in love with these books since I was 12 years old, and I got my first book for Christmas. I was hooked then, and I am still now, several years, books, movies and theme parks later. Once piece of me will always dream about this magical world, and Hogwarts will always be my ho

In [0]:
df_books = pd.read_csv(os.path.join(DIR, 'df_books.csv'))

In [0]:
print(df_reviews.shape)
print(df_books.shape)
print(df_reviews.dtypes)
print(df_books.dtypes)

(2080190, 2)
(276027, 6)
book_id         int64
review_text    object
dtype: object
average_rating    float64
book_id             int64
image_url          object
title              object
description        object
ratings_count       int64
dtype: object


In [0]:
df_reviews['review_text'] = df_reviews['review_text'].astype(str)
print(df_reviews.dtypes)

book_id         int64
review_text    object
dtype: object


In [0]:
result2 = df_reviews.merge(df_books, on='book_id', how='inner')

In [0]:
result2.shape

(275210, 7)

In [0]:
df_reviews = result2
export_csv = df_reviews.to_csv (r'/content/gdrive/My Drive/Goodreads/data/df_reviews.csv', index = None, header=True)

In [0]:
df_reviews_short = df_reviews[df_reviews.ratings_count > 1000] 
df_reviews_short = df_reviews_short.sort_values(['ratings_count'], ascending=[False])

In [0]:
df_reviews_short.head(10)

,book_id,review_text,average_rating,image_url,title,description,ratings_count
61921,2767052,I cracked and finally picked this up. Very enj...,4.34,https://images.gr-assets.com/books/1447303603m...,"The Hunger Games (The Hunger Games, #1)",Winning will make you famous.\nLosing means ce...,4899965
2,3,Tuve el gusto de leerlo antes de que saliera l...,4.45,https://images.gr-assets.com/books/1474154022m...,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter's life is miserable. His parents ...,4765497
4882,41865,"If I was 15, I would have LOVED this. It wasn'...",3.57,https://images.gr-assets.com/books/1361039443m...,"Twilight (Twilight, #1)",About three things I was absolutely positive.\...,3941381
335,2657,"Still relevant and beautiful., R.I.P. Ms. Harp...",4.26,https://images.gr-assets.com/books/1361975680m...,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,3255518
642,4671,I read this classic twice in high school. Its ...,3.89,https://images.gr-assets.com/books/1490528560m...,The Great Gatsby,"THE GREAT GATSBY, F. Scott Fitzgerald's third ...",2758812
114031,11870085,So.. well I know everyone's obsessed with this...,4.26,https://images.gr-assets.com/books/1360206420m...,The Fault in Our Stars,"There is an alternate cover edition .\n""I fel...",2429317
842,5907,Couldn't finish it. I was 5/6 of the way done ...,4.25,https://images.gr-assets.com/books/1372847500m...,The Hobbit,In a hole in the ground there lived a hobbit. ...,2099680
714,5107,"Certainly not his best, but it's Salinger so I...",3.79,https://images.gr-assets.com/books/1398034300m...,The Catcher in the Rye,The hero-narrator of The Catcher in the Ryeis ...,2086945
241,1885,This great classic's color paints vivid pictur...,4.25,https://images.gr-assets.com/books/1320399351m...,Pride and Prejudice,"""It is a truth universally acknowledged, that ...",2078406
150,960,"A quick read, and probably better than The Da ...",3.86,https://images.gr-assets.com/books/1303390735m...,"Angels & Demons (Robert Langdon, #1)",An ancient secret brotherhood.\nA devastating ...,2046499


In [0]:
export_csv = df_reviews_short.to_csv (r'/content/gdrive/My Drive/Goodreads/data/df_reviews_short.csv', index = None, header=True)

In [0]:
 df_reviews_short.head()

,book_id,review_text,average_rating,image_url,title,description,ratings_count
61921,2767052,I cracked and finally picked this up. Very enj...,4.34,https://images.gr-assets.com/books/1447303603m...,"The Hunger Games (The Hunger Games, #1)",Winning will make you famous.\nLosing means ce...,4899965
2,3,Tuve el gusto de leerlo antes de que saliera l...,4.45,https://images.gr-assets.com/books/1474154022m...,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter's life is miserable. His parents ...,4765497
4882,41865,"If I was 15, I would have LOVED this. It wasn'...",3.57,https://images.gr-assets.com/books/1361039443m...,"Twilight (Twilight, #1)",About three things I was absolutely positive.\...,3941381
335,2657,"Still relevant and beautiful., R.I.P. Ms. Harp...",4.26,https://images.gr-assets.com/books/1361975680m...,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,3255518
642,4671,I read this classic twice in high school. Its ...,3.89,https://images.gr-assets.com/books/1490528560m...,The Great Gatsby,"THE GREAT GATSBY, F. Scott Fitzgerald's third ...",2758812


In [59]:
df_reviews_short.head()

,book_id,review_text,average_rating,image_url,title,description,ratings_count
0,2767052,I cracked and finally picked this up. Very enj...,4.34,https://images.gr-assets.com/books/1447303603m...,"The Hunger Games (The Hunger Games, #1)",Winning will make you famous.\nLosing means ce...,4899965
1,3,Tuve el gusto de leerlo antes de que saliera l...,4.45,https://images.gr-assets.com/books/1474154022m...,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter's life is miserable. His parents ...,4765497
2,41865,"If I was 15, I would have LOVED this. It wasn'...",3.57,https://images.gr-assets.com/books/1361039443m...,"Twilight (Twilight, #1)",About three things I was absolutely positive.\...,3941381
3,2657,"Still relevant and beautiful., R.I.P. Ms. Harp...",4.26,https://images.gr-assets.com/books/1361975680m...,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,3255518
4,4671,I read this classic twice in high school. Its ...,3.89,https://images.gr-assets.com/books/1490528560m...,The Great Gatsby,"THE GREAT GATSBY, F. Scott Fitzgerald's third ...",2758812


In [0]:
df_reviews_short  = df_reviews_short.reset_index(drop=True)

In [65]:
len(df_reviews_short.review_text[1000])

654823